# Credit Risk Resampling Techniques

In [98]:
import warnings
warnings.filterwarnings('ignore')

In [99]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV and Perform Basic Data Cleaning

In [100]:
columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [101]:
# Load the data
file_path = Path('../Resources/LoanStats_2019Q1.csv.zip')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


# Label Encoding
Below I am label encoding (didn't see a use case for binary encoding) all of the columns so that I can scale

In [116]:
obj_labels = df.loc[:, df.dtypes == object].columns
#obj_labels = obj_labels.delete([2,6])
obj_labels

Index(['home_ownership', 'verification_status', 'issue_d', 'loan_status',
       'pymnt_plan', 'initial_list_status', 'next_pymnt_d', 'application_type',
       'hardship_flag', 'debt_settlement_flag'],
      dtype='object')

In [117]:
# need to categorize my data
from sklearn.preprocessing import LabelEncoder
home_ownership_encoder = LabelEncoder()
verification_status_encoder = LabelEncoder()
issue_d_encoder = LabelEncoder()
loan_status_encoder = LabelEncoder()
pymnt_plan_encoder = LabelEncoder()
init_list_status_encoder = LabelEncoder()
next_pymnt_d_encoder = LabelEncoder()
application_type_encoder = LabelEncoder()
hardship_flag_encoder = LabelEncoder()
debt_settlement_flag_encoder = LabelEncoder()
label_encoders = [
    home_ownership_encoder,
    verification_status_encoder,
    issue_d_encoder,
    loan_status_encoder,
    pymnt_plan_encoder,
    init_list_status_encoder,
    next_pymnt_d_encoder,
    application_type_encoder,
    hardship_flag_encoder,
    debt_settlement_flag_encoder
]

In [118]:
home_ownership_encoder.fit(df['home_ownership'])
verification_status_encoder.fit(df['verification_status'])
issue_d_encoder.fit(df["issue_d"])
loan_status_encoder.fit(df['loan_status'])
pymnt_plan_encoder.fit(df['pymnt_plan'])
init_list_status_encoder.fit(df['initial_list_status'])
next_pymnt_d_encoder.fit(df["next_pymnt_d"])
application_type_encoder.fit(df['application_type'])
hardship_flag_encoder.fit(df['hardship_flag'])
debt_settlement_flag_encoder.fit(df['debt_settlement_flag'])

LabelEncoder()

In [119]:
print(f"home_ownership : {list(home_ownership_encoder.classes_)}")
print(f"verification_status : {list(verification_status_encoder.classes_)}")
print(f"issue_d : {list(issue_d_encoder.classes_)}")
print(f"loan_status : {list(loan_status_encoder.classes_)}")
print(f"pymnt_plan : {list(pymnt_plan_encoder.classes_)}")
print(f"initial_list_status : {list(init_list_status_encoder.classes_)}")
print(f"next_pymnt_d : {list(next_pymnt_d_encoder.classes_)}")
print(f"application_type : {list(application_type_encoder.classes_)}")
print(f"hardship_flag : {list(hardship_flag_encoder.classes_)}")
print(f"debt_settlement : {list(debt_settlement_flag_encoder.classes_)}")

home_ownership : ['ANY', 'MORTGAGE', 'OWN', 'RENT']
verification_status : ['Not Verified', 'Source Verified', 'Verified']
issue_d : ['Feb-2019', 'Jan-2019', 'Mar-2019']
loan_status : ['high_risk', 'low_risk']
pymnt_plan : ['n']
initial_list_status : ['f', 'w']
next_pymnt_d : ['Apr-2019', 'May-2019']
application_type : ['Individual', 'Joint App']
hardship_flag : ['N']
debt_settlement : ['N']


In [120]:
ndf = df.drop(obj_labels, axis=1)
for x in range(len(obj_labels)):
    ndf[f"{obj_labels[x]}_trf"] = label_encoders[x].transform(df[obj_labels[x]])

In [121]:
new_columns = list(ndf.columns)
new_target = "loan_status_trf"

# Split the Data into Training and Testing

In [122]:
# Create our features
X = ndf[new_columns]

# Create our target
y = ndf[new_target]

In [123]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,home_ownership_trf,verification_status_trf,issue_d_trf,loan_status_trf,pymnt_plan_trf,initial_list_status_trf,next_pymnt_d_trf,application_type_trf,hardship_flag_trf,debt_settlement_flag_trf
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,1.812779,0.669994,0.805542,0.994958,0.0,0.876121,0.616839,0.139660,0.0,0.0
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.941313,0.719105,0.714932,0.070831,0.0,0.329446,0.486161,0.346637,0.0,0.0
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,1.000000,0.000000,0.000000,1.000000,0.0,1.000000,0.000000,0.000000,0.0,0.0
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,0.000000,0.0,0.0
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,3.000000,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,0.000000,0.0,0.0
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,3.000000,2.000000,2.000000,1.000000,0.0,1.000000,1.000000,1.000000,0.0,0.0


In [140]:
ndf.drop(['pymnt_plan_trf', 'hardship_flag_trf', 'debt_settlement_flag_trf'], axis=1, inplace=True)

In [141]:
# Check the balance of our target values
y.value_counts()

1    68470
0      347
Name: loan_status_trf, dtype: int64

In [142]:
# Create X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [143]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [144]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)

In [145]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [173]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({1: 51366, 0: 51366})

In [166]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [167]:
y_pred = model.predict(X_test)

In [168]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

0.6596586751752818

In [169]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[   71,    30],
       [ 6562, 10542]], dtype=int64)

In [181]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred, target_names=["high-risk", "low-risk"]))

                   pre       rec       spe        f1       geo       iba       sup

  high-risk       0.01      0.70      0.62      0.02      0.66      0.44       101
   low-risk       1.00      0.62      0.70      0.76      0.66      0.43     17104

avg / total       0.99      0.62      0.70      0.76      0.66      0.43     17205



### SMOTE Oversampling

In [183]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE
X_resampled_smote, y_resampled_smote = SMOTE(random_state=1, sampling_strategy=1.0).fit_resample(X_train, y_train)
Counter(y_resampled_smote)

Counter({1: 51366, 0: 51366})

In [184]:
# Train the Logistic Regression model using the resampled data
smote_model = LogisticRegression(solver='lbfgs', random_state=1)
smote_model.fit(X_resampled_smote, y_resampled_smote)

LogisticRegression(random_state=1)

In [185]:
y_pred_smote = smote_model.predict(X_test)

In [186]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_pred_smote)

0.6612934036621623

In [187]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred_smote)

array([[   65,    36],
       [ 5490, 11614]], dtype=int64)

In [188]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_smote, target_names=["high-risk", "low-risk"]))

                   pre       rec       spe        f1       geo       iba       sup

  high-risk       0.01      0.64      0.68      0.02      0.66      0.44       101
   low-risk       1.00      0.68      0.64      0.81      0.66      0.44     17104

avg / total       0.99      0.68      0.64      0.80      0.66      0.44     17205



# Undersampling

In this section, you will test an undersampling algorithms to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [182]:
# Resample the data using the ClusterCentroids resampler
from imblearn.under_sampling import ClusterCentroids
cc = ClusterCentroids(random_state=1)
X_resampled_cc, y_resampled_cc = cc.fit_resample(X_train, y_train)
Counter(y_resampled_cc)

Counter({0: 246, 1: 246})

In [190]:
# Train the Logistic Regression model using the resampled data
cc_model = LogisticRegression(solver='lbfgs', random_state=1, max_iter=2000)
cc_model.fit(X_resampled_cc, y_resampled_cc)
y_pred_cc = cc_model.predict(X_test)

In [191]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test,y_pred_cc)

0.6459608776593282

In [192]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred_cc)

array([[  78,   23],
       [8216, 8888]], dtype=int64)

In [193]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_cc, target_names=["high_risk", "low_risk"]))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.77      0.52      0.02      0.63      0.41       101
   low_risk       1.00      0.52      0.77      0.68      0.63      0.39     17104

avg / total       0.99      0.52      0.77      0.68      0.63      0.39     17205



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [194]:
# Resample the training data with SMOTEENN
from imblearn.combine import SMOTEENN

smoteenn_model = SMOTEENN(random_state=1)
X_resampled_smtn, y_resampled_smtn = smoteenn_model.fit_resample(X_train, y_train)
Counter(y_resampled_smtn)

Counter({0: 51361, 1: 46653})

In [196]:
# Train the Logistic Regression model using the resampled data
smtn_model = LogisticRegression(solver='lbfgs', random_state=1, max_iter=2000)
smtn_model.fit(X_resampled_smtn, y_resampled_smtn)

LogisticRegression(max_iter=2000, random_state=1)

In [197]:
y_pred_smtn = smtn_model.predict(X_test)

In [198]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_pred_smtn)

0.6899463040317129

In [199]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred_smtn)

array([[   77,    24],
       [ 6542, 10562]], dtype=int64)

In [200]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_smtn, target_names=["high_risk", "low_risk"]))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.76      0.62      0.02      0.69      0.48       101
   low_risk       1.00      0.62      0.76      0.76      0.69      0.46     17104

avg / total       0.99      0.62      0.76      0.76      0.69      0.46     17205

